# Face Recognition Project

Akhil Jariwala, Isaac Hales, Laurel Hales <br>
CS230 - Deep Learning <br>
v0.2 - 
2020/05/23

## Notebook Overview

This project takes photos, attempts to detect children (ages < 15) in the images, and then swaps each child face with a donor face of the same general age and gender.

This process can be broken down into three main sections:

* Face Detection
* Age and Gender Detection
* Face Swapping

Each section is completed using a network specifically built for the given task.

* Face Detection uses faced (https://github.com/iitzco/faced)
* Gender Detection is currently done using the process described here: https://data-flair.training/blogs/python-project-gender-age-detection/.
* We implemented age detection
* Face Swapping is achieved using fsgan (https://github.com/YuvalNirkin/fsgan)

### Import Base Libraries

In [ ]:
%matplotlib inline

In [ ]:
import os
import cv2
import sys
import random
import shutil
import math
import argparse
import pickle
import base64
from base64 import b64encode
import tensorflow as tf
# from google.colab.patches import cv2_imshow

import matplotlib.pyplot as plt



In [ ]:

base_dir = "/home/ubuntu/cs230/"

# Face Swapping requires a temporary directory
temp_dir = base_dir + "temp/"
# !mkdir "/home/ubuntu/cs230/temp/"

# Directory for gender and age detection files
gad_directory = base_dir + "gad/"

# Directory for test photos
test_photo_dir = base_dir + "familyPhotosCleanedBatch4/"

# Directory for fsgan
fsgan_dir = base_dir + 'fsgan/projects/'

# Donor Faces (from UTKFace dataset)
donor_dir = base_dir + "UTKFaceChildren/"

## Setup Code for Each Network

### Faced

In [ ]:
# Setup steps - just run once
#!git clone https://github.com/iitzco/faced.git

# Updated faced to use TensorFlow 2
# !git clone https://github.com/ihales/faced
# !pip install /home/ubuntu/cs230/faced

### fsgan

Acknowledgements - from the notebook in the fsgan repo: We thank Dr. Eyal Gruss, [wangchao0899](https://github.com/wangchao0899), [jjandnn](https://github.com/jjandnn), and [zhuhaozh](https://github.com/zhuhaozh) as well as the original author of the fsgan repository for helping with this demo.

In [ ]:
# Setup steps - just run once

# !sudo apt-get install ffmpeg
# !mkdir /home/ubuntu/cs230/fsgan/projects
# !mkdir /home/ubuntu/cs230/fsgan/data
# !pip install opencv-python ffmpeg-python yacs
# !cd /home/ubuntu/cs230/fsgan/projects
# !git clone https://github.com/YuvalNirkin/face_detection_dsfd
# !git clone https://github.com/YuvalNirkin/fsgan.git
# !conda install pytorch
# !pip install torchvision

sys.path += ['/usr/local/lib/python3.7/site-packages', base_dir + 'fsgan/projects']

#### Configure fsgan

In [ ]:
from fsgan.inference.swap import FaceSwapping
from fsgan.criterions.vgg_loss import VGGLoss

weights_dir = fsgan_dir + 'weights'
finetune_iterations = 800
seg_remove_mouth = True


detection_model = os.path.join(weights_dir, 'WIDERFace_DSFD_RES152.pth')
pose_model = os.path.join(weights_dir, 'hopenet_robust_alpha1.pth')
lms_model = os.path.join(weights_dir, 'hr18_wflw_landmarks.pth')
seg_model = os.path.join(weights_dir, 'celeba_unet_256_1_2_segmentation_v2.pth')
reenactment_model = os.path.join(weights_dir, 'nfv_msrunet_256_1_2_reenactment_v2.1.pth')
completion_model = os.path.join(weights_dir, 'ijbc_msrunet_256_1_2_inpainting_v2.pth')
blending_model = os.path.join(weights_dir, 'ijbc_msrunet_256_1_2_blending_v2.pth')
criterion_id_path = os.path.join(weights_dir, 'vggface2_vgg19_256_1_2_id.pth')
criterion_id_path = os.path.join(weights_dir, 'vggface2_vgg19_256_1_2_id.pth')
criterion_id = VGGLoss(criterion_id_path)


face_swapping = FaceSwapping(
    detection_model=detection_model, pose_model=pose_model, lms_model=lms_model,
    seg_model=seg_model, reenactment_model=reenactment_model,
    completion_model=completion_model, blending_model=blending_model,
    criterion_id=criterion_id,
    finetune=True, finetune_save=True, finetune_iterations=finetune_iterations,
    seg_remove_mouth=finetune_iterations, batch_size=16, seg_batch_size=48,
    encoder_codec='mp4v')

#### Function to call fsgan and return a OpenCV image with swaped face

In [ ]:


def setup_donor_face(age, gender):
    """ Given and age classification and gender, return a donor face"""
  
    # convert gender encodings
    gender = 1 if gender == "f" else 0

    # filter through possible faces
    filenames = os.listdir(donor_dir)

    donor_file = random.choice([x for x in filenames if int(x.split("_")[1]) == gender])
    shutil.copyfile(donor_dir + donor_file, temp_dir + "source_image.jpg")

def swap_face(original_face, age_and_gender):
    """ Given an image, age classification, and gender, replaces the face with a donor face.

    Arguments:
    original_face -- an OpenCV image to use for swapping
    age_and_gender -- a tuple with an integer age classification and a gender ("m" or "f")

    Returns:
    replacement_image -- an OpenCV image to use as a replacement in the original picture
    """
    age = age_and_gender[0]
    gender = age_and_gender[1]

    # Quit early if not a minor
    if age > 15:
        return original_face


    # Start by saving the image to disk
    target_image_path = temp_dir + "target_image.jpg"
    cv2.imwrite(target_image_path, original_face)
    # Select a source image at random from directory of face donors
    setup_donor_face(age, gender)
    source_image_path = temp_dir + "source_image.jpg"

    # Swap the faces
    finetune = True
    output_tmp_path = temp_dir + 'output_tmp.mp4'
    face_swapping(source_image_path, target_image_path, output_tmp_path,
                'longest', 'longest', finetune)

    vidcap = cv2.VideoCapture(output_tmp_path)
    success, image = vidcap.read()

    # Remove Temp Files
    os.remove(output_tmp_path)
    os.remove(target_image_path)
    os.remove(source_image_path)
    shutil.rmtree(temp_dir + "target_image/",)
    shutil.rmtree(temp_dir + "source_image/",)


    return image
  

### Age and Gender Detection

In [ ]:
# # Filenames and data for all the existing models and weights

age_model = "/home/ubuntu/cs230/saved_models/final_model"

genderProto="gender_deploy.prototxt"
genderModel="gender_net.caffemodel"

MODEL_MEAN_VALUES=(78.4263377603, 87.7689143744, 114.895847746)

# List of options for output ages and genders
genderList=['m','f']

# Model definitions
genderNet=cv2.dnn.readNet(gad_directory + genderModel, gad_directory + genderProto)
ageNet = tf.keras.models.load_model(age_model, compile = False)

In [ ]:
def get_age_and_gender(face, padding=20):
    IMG_SIZE=224
    
    blob=cv2.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=False)
    genderNet.setInput(blob)
    genderPreds=genderNet.forward()
    gender=genderList[genderPreds[0].argmax()]

    ## This is where I will include our age detector

    newImage = face.copy()
    newImageRecolored = cv2.cvtColor(newImage, cv2.COLOR_BGR2RGB)
    
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(newImageRecolored, tf.float32)

    # resize the image to the desired size.
    formatted_image = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    formatted_image = tf.reshape(formatted_image, [1, IMG_SIZE, IMG_SIZE, 3])
    
    age = ageNet.predict(formatted_image, steps=1)[0][0]

    return (age, gender)


### Initialize FaceDetector

In [ ]:
from faced import FaceDetector
face_detector = FaceDetector()
thresh = 0.6

### Test Sample Classifications

In [ ]:
# Thanks to https://stackoverflow.com/a/54037086 for helping out with this one

def byte_me(img):
    _, buffer_img = cv2.imencode('.jpg', img)
    data = base64.b64encode(buffer_img).decode("ascii")
    return data

In [ ]:
photo_count = 0
photo_facts_list = []

photo_list = os.listdir(test_photo_dir)

for photoName in photo_list:
    photo_count += 1
    print("Photo: ", photo_count)
    ###
    this_photo={"id": photo_count}

    testImagePath = test_photo_dir + photoName
    testImage = cv2.imread(testImagePath)

    newImage = testImage.copy()
    ###
    this_photo["before"] = byte_me(newImage)
    this_photo["faces"] = []


    newImageRecolored = cv2.cvtColor(newImage, cv2.COLOR_BGR2RGB)

    # Receives RGB numpy image (HxWxC) and
    # returns (x_center, y_center, width, height, prob) tuples. 
    bboxes = face_detector.predict(newImageRecolored, thresh = thresh)

    croppedFaces = []

    #Set padding pixels for each photo to improve age detection algorithm
    padding = 50

    #Collect image width and height as maximums for bounding box x and y-coordinates
    imageWidth = int(newImage.shape[1])
    imageHeight = int(newImage.shape[0])

    #Create a list for the adjusted face bounding boxes
    faceBoundingBoxes = []

    for box in bboxes:
        # Extract x and y coordinates
        xc, yc, wi, he, p = box
        x1 = max(int(xc - wi//2 - padding),0)
        x2 = min(int(xc + wi//2 + padding), imageWidth - 1)
        x2 -= (x2-x1)%2
        y1 = max(int(yc - he//2 - padding), 0)
        y2 = min(int(yc + he//2 + padding), imageHeight - 1)
        y2 -= (y2-y1)%2

        paddedBox = [x1, x2, y1, y2]

        faceBoundingBoxes.append(paddedBox)

        #Add cropped face to list
        croppedFace = newImage[y1:y2, x1:x2]
        croppedFaces.append(croppedFace)

    #Create a list for the swapped faces
    swappedFaces = []

    for face in croppedFaces:
        this_face={"before": byte_me(face)}

        # collect age and gender of face
        age, gender = get_age_and_gender(face)

        ###
        this_face['age'] = age
        this_face['gender'] = gender

        # Generate swapped face
        swapped_face = swap_face(face, (age,gender))


        ###
        this_face['after'] = byte_me(swapped_face)
        this_photo['faces'].append(this_face)


        # Add swapped face to new list of faces
        swappedFaces.append(swapped_face)

  
    #Combine the faceBoundingBoxes and swappedFaces list to identify swappings to perform
    swappingsToDo = list(zip(faceBoundingBoxes,swappedFaces))

    for swapItem in swappingsToDo:
        faceBoundingBox, replacementFace = swapItem
        x1, x2, y1, y2 = faceBoundingBox

        newImage[y1:y2, x1:x2] = replacementFace[0:y2-y1, 0:x2-x1]

      
    this_photo['after'] = byte_me(newImage)
    photo_facts_list.append(this_photo)

In [ ]:
# Save the output to analyze later
pickle.dump(photo_facts_list, open("photo_facts.p", "wb"))